In [287]:
import pandas as pd
import numpy as np

In [288]:
from PIL import Image, ImageFont, ImageDraw

In [348]:
def toFixed(numObj, digits=3):
    return f"{numObj:.{digits}f}2"

In [289]:
def foo(item):
    if (item == 255):
        return 1
    return 0

In [290]:
def weight_of_black(image, arr):
    area = (arr[0], arr[1], arr[2], arr[3])
    symbol = image.crop(area)
    width, height = symbol.size
    sum_ = 0
    for x in range(width):
        for y in range(height):
            sum_ += foo(symbol.getpixel((x,y)))
    return width * height - sum_, (width * height - sum_) / (width * height)

In [291]:
def coords_of_center_of_gravity(image, arr):
    area = (arr[0], arr[1], arr[2], arr[3])
    symbol = image.crop(area)
    width, height = symbol.size
    sum1 = 0
    sum2 = 0
    for x in range(width):
        for y in range(height):
            sum1 += (x + 1) * (1 - foo(symbol.getpixel((x,y))))
    for x in range(width):
        for y in range(height):
            sum2 += (y + 1) * (1 - foo(symbol.getpixel((x,y)))) 
    weight = weight_of_black(image, arr)
    return sum1 / weight[0] , sum2 / weight[0]

In [292]:
def norm_coords_of_center_of_gravity(image, arr):
    area = (arr[0], arr[1], arr[2], arr[3])
    symbol = image.crop(area)
    width, height = symbol.size
    sum1 = 0
    sum2 = 0
    for x in range(width):
        for y in range(height):
            sum1 += (x + 1)* (1 - foo(symbol.getpixel((x,y))))
    for x in range(width):
        for y in range(height):
            sum2 += (y + 1) * (1 - foo(symbol.getpixel((x,y)))) 
    weight = weight_of_black(image, arr)[0]
    return ((sum1 / weight) - 1) / (width - 1),  ((sum2 / weight) - 1) / (height - 1)

In [293]:
def norm_axial_moments_of_inertia(image, arr):
    area = (arr[0], arr[1], arr[2], arr[3])
    symbol = image.crop(area)
    width, height = symbol.size
    sum1 = 0
    sum2 = 0
    center = coords_of_center_of_gravity(image, arr)
    for x in range(width):
        for y in range(height):
            sum1 += ((1 - foo(symbol.getpixel((x,y)))) * ((y + 1 - center[1]) ** 2))
    for x in range(width):
        for y in range(height):
            sum2 += ((1 - foo(symbol.getpixel((x,y)))) * ((x + 1 - center[0]) ** 2))
    return sum1 / ((width ** 2) + (height ** 2)), sum2 / ((width ** 2) + (height ** 2))

In [294]:
df_data = pd.read_csv("signs.csv", sep=';')

In [392]:
image = Image.open("cropped_text2.bmp")

In [393]:
image.show()

In [394]:
with open("coords2.txt") as file:
    lines = file.readlines()

In [395]:
arrs = []

In [396]:
for line in lines:
    string = line.split(' ')
    string.pop()
    arr = []
    for item in string:
        arr.append(int(item))
    arrs.append(arr)

In [397]:
arrs

[[0, 7, 18, 26],
 [23, 0, 40, 33],
 [42, 7, 59, 33],
 [62, 0, 80, 26],
 [83, 7, 97, 26],
 [99, 0, 114, 33],
 [116, 7, 131, 33],
 [136, 0, 153, 26],
 [157, 7, 161, 26],
 [166, 7, 181, 26],
 [183, 0, 200, 26],
 [204, 7, 219, 33],
 [224, 7, 240, 26],
 [242, 0, 256, 33],
 [257, 7, 275, 26],
 [277, 7, 301, 26],
 [305, 7, 322, 33],
 [324, 7, 345, 26],
 [347, 7, 360, 26],
 [364, 7, 379, 26],
 [384, 7, 405, 33],
 [406, 7, 423, 33],
 [427, 7, 449, 33],
 [453, 7, 479, 26]]

In [398]:
df = pd.DataFrame({'A' : [weight_of_black(image, arr)[1] for arr in arrs],
                   'B' : [norm_coords_of_center_of_gravity(image, arr)[0] for arr in arrs],
                   'C' : [norm_coords_of_center_of_gravity(image, arr)[1] for arr in arrs],
                   'D' : [norm_axial_moments_of_inertia(image, arr)[0] for arr in arrs],
                   'E' : [norm_axial_moments_of_inertia(image, arr)[1] for arr in arrs]})
    

In [400]:
def p_measure(d):
    return 1/(1 + 0.8 * d)

In [401]:
class Predictor:
    def __init__(self, X, y):
        self.X = X
        self.y = y
    def predict(self, X):
        predictions = []
        for vls in X:
            d = 0
            for index, value in enumerate(vls):
                d += (self.X[:, index] - value)**2
            d=np.sqrt(d)
            idx = np.argsort(d)
            predictions.append([(self.y[index], float(toFixed(p_measure(d[index]), 5))) for index in idx])
        return predictions

In [402]:
columns = ['symbol', 'B', 'E', 'F', 'I', 'J']

In [403]:
train = df_data[columns]

In [404]:
X_train = train.drop(columns = ['symbol']).values

In [406]:
y_train = train['symbol'].values

In [407]:
y_train

array(['α', 'β', 'γ', 'δ', 'ε', 'ζ', 'η', 'θ', 'ι', 'κ', 'λ', 'μ', 'ν',
       'ξ', 'ο', 'π', 'ρ', 'σ', 'τ', 'υ', 'φ', 'χ', 'ψ', 'ω'],
      dtype=object)

In [408]:
X_test = df.values

In [409]:
mean_ = X_train.mean(axis=0)
std_ = X_train.std(axis=0)

X_train = (X_train - mean_) / std_
X_test = (X_test - mean_) / std_

In [410]:
max_ = X_train.max(axis=0)
min_ = X_train.min(axis=0)

X_train = (X_train - min_) / (max_ - min_)
X_test = (X_test - min_) / (max_ - min_)

In [411]:
predictor = Predictor(X_train, y_train)

In [412]:
predictions = predictor.predict(X_test)

In [414]:
with open("predictions2.txt", 'w', encoding = "utf-8") as f:
    for item in predictions:
        f.write(str(item) + "\n")

In [390]:
string = ''
for item in predictions:
    string += item[0][0]

In [391]:
string

'πλαγχθπμημιτμοιπζιμμονητολιμθμον'